In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy
import quandl
from stockAnalysisTools import numToString
from time import time

#### The cell below will be used to pull all the stock data we want and make a dictionary for accessing it

In [2]:
quandl.ApiConfig.api_key = "d1JcayaRuvKSaq7GaEHs"

import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import re

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
html = urllib.request.urlopen(url, context=ctx).read()
soup = BeautifulSoup(html, 'html.parser')

# soup
ticker_list = list()
for tag in soup('a'):
#     print(re.search(r'NyseSymbol|\S+}', soup))
    link = str(tag.get('href', None))
#     print(type(link))
    if "https://www.nyse.com/quote/XNYS" in link:
        ticker_list.append(link.rsplit(':')[-1])
    elif "http://www.nasdaq.com/symbol/" in link:
        ticker_list.append(link.rsplit('/')[-1])
    

# print(ticker_list)
# ticker_list.remove('QECx02')
# print(len(ticker_list))
ticker_list = list(map(lambda x: x.replace('.', '_') if '.' in x else x, ticker_list))
# ticker_list

In [4]:
startDate = '2001-07-01'
endDate = '2018-07-01'
finData = dict()
t0 = time()
print("Couldn't retrieve data for the stocks below, they may not have data available on Quandl.com, \
which is where the data is fetched from")
for ticker in ticker_list:
    flag = 0
    try:
        finData[ticker] = quandl.get("WIKI/"+ticker)["Adj. Close"].loc[startDate:endDate]
        flag = 1
    except:
        if flag == 0:
            print(ticker)
        
print("Run time:", round(time()-t0, 3)/60, "min")

Couldn't retrieve data for the stocks below, they may not have data available on Quandl.com, which is where the data is fetched from
EVRG
JEF
Run time: 37.90321666666667 min


In [5]:
#     #All data will be normalized to start on the indecies of apple, so that when returns are calculated, the math can be done properly
aaplIndecies = finData["aapl"].index
    
def createRollingDF(key):
    rollingStat = finData[key].rolling(window=30,center=False).mean().dropna()
    rollingStd = finData[key].rolling(window=30,center=False).std().dropna()

    rollingStat = rollingStat.reset_index()

    rollingStat = rollingStat.rename(index=str, columns={"Adj. Close": "Rolling Mean"})
    rollingStat["Std"] = rollingStd.values
    rollingStat["Upper Band"] = rollingStat.apply(lambda row: row["Rolling Mean"] + 2*row["Std"], axis=1)
    rollingStat["Lower Band"] = rollingStat.apply(lambda row: row["Rolling Mean"] - 2*row["Std"], axis=1)

    rollingStat = rollingStat.set_index("Date")

    rollingStat["Adj. Close"] = finData[key]
    return rollingStat
    
def determineBuySell(df):
    df = df.reset_index()
    bd = df.apply(lambda row: (row["Rolling Mean"] - row["Adj. Close"])/row["Std"] \
                  if row["Adj. Close"] < row["Lower Band"] else 0, axis=1)
    sd = df.apply(lambda row: (row["Adj. Close"] - row["Rolling Mean"])/row["Std"] \
                  if row["Adj. Close"] > row["Upper Band"] else 0, axis=1)
 
    ret_df = pd.DataFrame()
    ret_df["Date"] = df["Date"]
    df = df.set_index("Date")
 
    ret_df["Position"] = bd + sd
    ret_df = ret_df.set_index("Date")
    ret_df["Buy Day"] = ret_df.apply(lambda row: 1 if row["Position"] > 0 else 0, axis=1)
    ret_df["Sell Day"] = ret_df.apply(lambda row: 1 if row["Position"] < 0 else 0, axis=1)
#     ret_df["Adj. Close"] = df["Adj. Close"]

    ret_df["Returns"] = df["Adj. Close"].pct_change().shift(-1).dropna()
#     ret_df["Returns"] = returns *  ret_df["Position"]
#     ret_df["Returns"] = ret_df["Returns"].dropna()
#     print(aaplIndecies)
    ret_df = ret_df.reindex(aaplIndecies)
    ret_df = ret_df.fillna(0)
    ret_df["Returns"][-1] = np.nan
    return ret_df
    

In [6]:
rollingDFs = dict()
t0 = time()
keys_to_rm = list()
for key in finData.keys():
    flag = 0
    try:
        rollingDFs[key] = createRollingDF(key)
        flag = 1
    except:
        if flag == 0:
            keys_to_rm.append(key)
            
print("Run time:", round(time()-t0, 3)/60, "min")

#Keys for some stocks end up getting removed because they cause errors due to
#not enough market data being available to calculate a rolling stat over a 30 day period
print("Couldn't calculate rolling data for the stocks below, so they are being removed from the findData dictionary:")
for key in keys_to_rm:
    print(key)
#     finData.pop(key, None)

Run time: 12.358433333333332 min
Couldn't calculate rolling data for the stocks below, so they are being removed from the findData dictionary:
bkng
CBRE
WELL


In [7]:
returnsDFs = dict()
t0 = time()
keys_to_rm = list()
for key in rollingDFs.keys():
    flag = 0
    try:
        returnsDFs[key] = determineBuySell(rollingDFs[key])
        flag = 1
    except:
        if flag == 0:
            keys_to_rm.append(key)
            
print("Run time:", round(time()-t0, 3)/60, "min")

#Keys for some stocks end up getting removed because they cause errors due to
#not enough market data being available to calculate a rolling stat over a 30 day period
print("Couldn't calculate return data for the stocks below, so they are being removed from the findData dictionary:")
for key in keys_to_rm:
    print(key)
    finData.pop(key, None)

Run time: 10.607983333333333 min
Couldn't calculate return data for the stocks below, so they are being removed from the findData dictionary:


In [8]:
dates = dict()
comparison_df = pd.DataFrame()
flag = True
t0 = time()
for key in returnsDFs.keys():
#     print(key)
#     print(returnsDFs[key])
#     break
    if flag is True:
        comparison_df = returnsDFs[key]["Position"].to_frame()
        comparison_df = comparison_df.rename(index=str, columns={"Position": key})
        flag = False 
    else:
#         print(key)
        comparison_df[key] = returnsDFs[key]["Position"]
#         print(comparison_df)
#         print(returnsDFs[key]["Position"].cumsum())
#         break
#         comparison_df = pd.merge(left = comparison_df, right = returnsDFs[key]["Position"].to_frame(), how = 'inner', on = "Date")
#         comparison_df
#         break

comparison_df.fillna(0)
print("Run time:", round(time()-t0, 3), "s")

Run time: 0.821 s


In [9]:
t0 = time()
transpose_df = comparison_df.T
for date in transpose_df:
    dates[date] = transpose_df[date].nlargest(50).index.tolist()
print("Run time:", round(time()-t0, 3), "s")
# print(dates)

Run time: 2.869 s


In [10]:
def determineTop50(df, key):
    df = df.reset_index()
    df["Position"] = df.apply(lambda row: row["Buy Day"] + (-1*row["Sell Day"]) if key in dates[str(row["Date"])] else 0,  axis=1)
    df = df.set_index("Date")
    return df["Position"]

In [11]:
t0 = time()
for key in returnsDFs.keys():
    returnsDFs[key]["Position"] = determineTop50(returnsDFs[key], key)
print("Run time:", round(time()-t0, 3)/60, "min")

t0 = time()
flag = True
for key in returnsDFs.keys():
    d = returnsDFs[key]
    if flag is True:
        ret = d["Position"] * d["Returns"]
        flag = False
    else:
        ret += d["Position"] * d["Returns"]
        
print("Run time:", round(time()-t0, 3), "s")

Run time: 4.680466666666666 min
Run time: 0.326 s


## Sharpe Calculated

In [12]:
ret.mean()/ret.std()

0.03151458791930259

## Beta Calculated

In [ ]:
sp500 = quandl.get("MULTPL/SP500_REAL_PRICE_MONTH").loc[startDate:endDate]

ret.corr(sp500["Value"])